# 第5章: 係り受け解析

[日本語Wikipedia](https://ja.wikipedia.org/)の「[人工知能](https://ja.wikipedia.org/wiki/%E4%BA%BA%E5%B7%A5%E7%9F%A5%E8%83%BD)」に関する記事からテキスト部分を抜き出したファイルが[ai.ja.zip](https://nlp100.github.io/data/ai.ja.zip)に収録されている． この文章を[CaboCha](https://taku910.github.io/cabocha/)や[KNP](http://nlp.ist.i.kyoto-u.ac.jp/index.php?KNP)等のツールを利用して係り受け解析を行い，その結果をai.ja.txt.parsedというファイルに保存せよ．このファイルを読み込み，以下の問に対応するプログラムを実装せよ．

In [1]:
;wget https://nlp100.github.io/data/ai.ja.zip

--2020-12-02 16:47:58--  https://nlp100.github.io/data/ai.ja.zip
nlp100.github.io (nlp100.github.io) をDNSに問いあわせています... 185.199.108.153, 185.199.109.153, 185.199.110.153, ...
nlp100.github.io (nlp100.github.io)|185.199.108.153|:443 に接続しています... 接続しました。
HTTP による接続要求を送信しました、応答を待っています... 200 OK
長さ: 17516 (17K) [application/zip]
`ai.ja.zip' に保存中

     0K .......... .......                                    100% 14.6M=0.001s

2020-12-02 16:47:59 (14.6 MB/s) - `ai.ja.zip' へ保存完了 [17516/17516]



In [2]:
;unzip ai.ja.zip

Archive:  ai.ja.zip
  inflating: ai.ja.txt               
  inflating: readme.ai.ja.md         


In [8]:
;cabocha -f1 -o ai.ja.txt.parsed ai.ja.txt

In [9]:
;wc -l ./ai.ja.txt.parsed

   11744 ./ai.ja.txt.parsed


In [ ]:
;head 

**40. 係り受け解析結果の読み込み（形態素）**<br>
形態素を表すクラスMorphを実装せよ．このクラスは表層形（surface），基本形（base），品詞（pos），品詞細分類1（pos1）をメンバ変数に持つこととする．さらに，係り受け解析の結果（ai.ja.txt.parsed）を読み込み，各文をMorphオブジェクトのリストとして表現し，冒頭の説明文の形態素列を表示せよ．

 [ここ](https://qiita.com/yamaru/items/48dcc527f433c22e0af9)を参考にしました


In [1]:
module morph
    export Morph

    struct Morph
        surface
        base
        pos
        pos1
    end
    function Morph(morph)
        surface, attr=split(morph,"\t")
        attr=split(attr, ",")
        base=attr[7]
        pos=attr[1]
        pos1=attr[2]
        return Morph(surface,base,pos,pos1)
    end
end


Main.morph

In [4]:
using .morph
filename="./ai.ja.txt.parsed"

global sentences=[]
global morphs=[]
open(filename, "r") do f
    for line in eachline(f)
        if line[1]=='*'
            continue
        elseif line!="EOS"
            push!(morphs,Morph(line))
        else
            push!(sentences,morphs)
            global morphs=[]
        end
    end
end

for m in sentences[3]
            println(m)
end

Morph("人工", "人工", "名詞", "一般")
Morph("知能", "知能", "名詞", "一般")
Morph("（", "（", "記号", "括弧開")
Morph("じん", "じん", "名詞", "一般")
Morph("こうち", "こうち", "名詞", "一般")
Morph("のう", "のう", "助詞", "終助詞")
Morph("、", "、", "記号", "読点")
Morph("、", "、", "記号", "読点")
Morph("AI", "*", "名詞", "一般")
Morph("〈", "〈", "記号", "括弧開")
Morph("エーアイ", "*", "名詞", "固有名詞")
Morph("〉", "〉", "記号", "括弧閉")
Morph("）", "）", "記号", "括弧閉")
Morph("と", "と", "助詞", "格助詞")
Morph("は", "は", "助詞", "係助詞")
Morph("、", "、", "記号", "読点")
Morph("「", "「", "記号", "括弧開")
Morph("『", "『", "記号", "括弧開")
Morph("計算", "計算", "名詞", "サ変接続")
Morph("（", "（", "記号", "括弧開")
Morph("）", "）", "記号", "括弧閉")
Morph("』", "』", "記号", "括弧閉")
Morph("という", "という", "助詞", "格助詞")
Morph("概念", "概念", "名詞", "一般")
Morph("と", "と", "助詞", "並立助詞")
Morph("『", "『", "記号", "括弧開")
Morph("コンピュータ", "コンピュータ", "名詞", "一般")
Morph("（", "（", "記号", "括弧開")
Morph("）", "）", "記号", "括弧閉")
Morph("』", "』", "記号", "括弧閉")
Morph("という", "という", "助詞", "格助詞")
Morph("道具", "道具", "名詞", "一般")
Morph("を", "を", "助詞", "格助詞")
Morph("用い",

**41. 係り受け解析結果の読み込み（文節・係り受け）**<br>
40に加えて，文節を表すクラスChunkを実装せよ．このクラスは形態素（Morphオブジェクト）のリスト（morphs），係り先文節インデックス番号（dst），係り元文節インデックス番号のリスト（srcs）をメンバ変数に持つこととする．さらに，入力テキストの係り受け解析結果を読み込み，１文をChunkオブジェクトのリストとして表現し，冒頭の説明文の文節の文字列と係り先を表示せよ．本章の残りの問題では，ここで作ったプログラムを活用せよ．

In [8]:
module chunk
    export Chunk

    struct Chunk
        morphs
        dst
        srcs
    end
    function Chunk(morphs_, dst_)
        morphs=morphs_
        dst=dst_
        srcs=[]
        return Chunk(morphs, dst, srcs)
    end
end
module sentence
    export Sentence
    
    struct Sentence
        chunks
    end
    function Chunk(chunks_)
        chunks=chunks_
        for (i ,chunk) in enumerate(chunks)
            if chunk.dst!=-1
                push!(chunks[chunk.dst].srcs,i)
            end
        end
    end
end

Main.sentence

In [20]:
using .chunk
using .sentence
global sentences=[]
global chunks=[]
global morphs=[]
global dst=0
open(filename, "r") do f
    for line in eachline(f)
        if line[1]=='*'
            if length(morphs)>0
                push!(chunks, Chunk(morphs, dst))
                global morphs=[]
            end
            global dst=split(line," ")[3]
            global dst=parse(Int,replace.(dst,['D'=>""])[1])
        elseif line!="EOS"
            push!(morphs,Morph(line))
        else
            push!(chunks,Chunk(morphs, dst))
            push!(sentences, Sentence(chunks))
            global morphs=[]
            global chunks=[]
        end
    end
end

for chunk in sentences[3].chunks
    println([morph.surface for morph in chunk.morphs],chunk.dst, chunk.srcs)
end

SubString{String}["人工", "知能"]17Any[]
SubString{String}["（", "じん", "こうち", "のう", "、", "、"]17Any[]
SubString{String}["AI"]3Any[]
SubString{String}["〈", "エーアイ", "〉", "）", "と", "は", "、"]17Any[]
SubString{String}["「", "『", "計算"]5Any[]
SubString{String}["（", "）", "』", "という"]9Any[]
SubString{String}["概念", "と"]9Any[]
SubString{String}["『", "コンピュータ"]8Any[]
SubString{String}["（", "）", "』", "という"]9Any[]
SubString{String}["道具", "を"]10Any[]
SubString{String}["用い", "て"]12Any[]
SubString{String}["『", "知能", "』", "を"]12Any[]
SubString{String}["研究", "する"]13Any[]
SubString{String}["計算", "機", "科学"]14Any[]
SubString{String}["（", "）", "の"]15Any[]
SubString{String}["一", "分野", "」", "を"]16Any[]
SubString{String}["指す"]17Any[]
SubString{String}["語", "。"]34Any[]
SubString{String}["「", "言語", "の"]20Any[]
SubString{String}["理解", "や"]20Any[]
SubString{String}["推論", "、"]21Any[]
SubString{String}["問題", "解決", "など", "の"]22Any[]
SubString{String}["知的", "行動", "を"]24Any[]
SubString{String}["人間", "に"]24Any[]
SubString{String}